# Подготовка ноутбука 

## Пробрасываем magic methods

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

## Imports

In [ ]:
import pandas as pd 
import numpy as np 
from data.prepare_data import PrepareData
from dotenv import load_dotenv
import os 
from pathlib import Path
from warnings import filterwarnings 
import torch 
import torchvision
from torch.utils.data import DataLoader
from IPython.display import display
import pytesseract
import shutil
try:
    from PIL import Image
except ImportError:
     import Image
import cv2
import requests
import json
import os
from datetime import datetime
import csv

## Нужные переменные 

In [37]:
ROOT_DIR = Path('../../')
load_dotenv()
filterwarnings(action='ignore')

# Загрузка датафрейма

In [ ]:
class MapillaryClient:
    def __init__(self, access_token):
        self.access_token = access_token
        self.base_url = "https://graph.mapillary.com"
        self.headers = {"Authorization": f"Bearer {access_token}"}
    
    def get_images_in_bbox(self, bbox, max_results=1000):
        """
        bbox: [min_lon, min_lat, max_lon, max_lat]
        """
        all_images = []
        url = f"{self.base_url}/images"
        
        params = {
            'fields': 'id,geometry,compass_angle,captured_at,altitude,sequence,creator',
            'access_token': self.access_token,
            'limit': min(max_results, 2000),  # Максимальный лимит API
            'bbox': ','.join(map(str, bbox))
        }
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if 'data' in data:
                return data['data']
            else:
                print("Неожиданный формат ответа:", data)
                return []
                
        except requests.exceptions.RequestException as e:
            print(f"Ошибка запроса: {e}")
            return []
    
    def save_metadata_to_csv(self, images, filename):
        """Сохранение метаданных в CSV"""
        if not images:
            print("Нет данных для сохранения")
            return
        
        os.makedirs('mapillary_data', exist_ok=True)
        filepath = f'mapillary_data/{filename}'
        
        with open(filepath, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['id', 'latitude', 'longitude', 'captured_at', 
                         'compass_angle', 'altitude', 'sequence_id', 'creator']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            for img in images:
                geometry = img.get('geometry', {})
                writer.writerow({
                    'id': img.get('id'),
                    'latitude': geometry.get('coordinates', [0, 0])[1],
                    'longitude': geometry.get('coordinates', [0, 0])[0],
                })
        
        print(f"Метаданные сохранены в {filepath}")
    
    def download_images_batch(self, images, download_folder=ROOT_DIR / "data/processed_data/moscow_mapillary"):
        """Пакетная загрузка изображений"""
        os.makedirs(download_folder, exist_ok=True)
        
        downloaded = 0
        for i, img in enumerate(images):
            try:
                image_id = img.get('id')
                if self.download_single_image(image_id, f"{download_folder}/{image_id}.jpg"):
                    downloaded += 1
                
                # Ограничение скорости чтобы не перегружать API
                if i % 10 == 0:
                    print(f"Загружено {downloaded}/{len(images)} изображений")
                    
            except Exception as e:
                print(f"Ошибка при загрузке {image_id}: {e}")
                continue
        
        print(f"Загрузка завершена. Успешно: {downloaded}/{len(images)}")
    
    def download_single_image(self, image_id, save_path):
        """Загрузка одного изображения"""
        try:
            # Получаем детальную информацию с URL изображения
            url = f"{self.base_url}/{image_id}"
            params = {
                'fields': 'thumb_2048_url',
                'access_token': self.access_token
            }
            
            response = requests.get(url, params=params)
            data = response.json()
            
            image_url = data.get('thumb_2048_url')
            if image_url:
                img_response = requests.get(image_url)
                with open(save_path, 'wb') as f:
                    f.write(img_response.content)
                return True
            else:
                print(f"URL изображения не найден для {image_id}")
                return False
                
        except Exception as e:
            print(f"Ошибка при загрузке {image_id}: {e}")
            return False

# Пример использования
def main():
    # Инициализация клиента
    ACCESS_TOKEN = os.getenv('API_MAPILLARY_KEY')
    client = MapillaryClient(ACCESS_TOKEN)

    # Bounding box для центра Москвы
    moscow_bbox = [37.149337, 55.9702540, 37.161783, 55.9770450]
    
    print("Получение изображений Mapillary для Москвы...")
    images = client.get_images_in_bbox(moscow_bbox, max_results=500)
    
    if images:
        print(f"Найдено {len(images)} изображений")
        
        # Сохраняем метаданные
        client.save_metadata_to_csv(images, ROOT_DIR / "data/processed_data/moscow_mapillary.csv")
        
        # Загружаем изображения (опционально)
        download_choice = input("Загрузить изображения? (y/n): ")
        if download_choice.lower() == 'y':
            client.download_images_batch(images)  # Первые 50 для примера
    else:
        print("Изображения не найдены")

if __name__ == "__main__":
    main()

NameError: name 'ROOT_DIR' is not defined